In [4]:
import pandas as pd
import spotipy as sp
from dotenv import load_dotenv

# Load credentials from environment variables
load_dotenv()
auth_manager = sp.oauth2.SpotifyClientCredentials()
spotify = sp.Spotify(auth_manager=auth_manager)

In [23]:
def get_all_user_playlist_ids(user):
    """Gets the ids for all user playlists built by a given user."""
    
    playlists = spotify.user_playlists(user)
    playlist_ids = []
    # iterate for as many pages that exist
    while playlists:
        for playlist in playlists['items']:
            playlist_ids.append(playlist['id'])
        if playlists['next']:
            playlists = spotify.next(playlists)
        else:
            playlists = None
    return playlist_ids


def get_all_playlist_tracks(playlist_ids, limit_per_playlist=1, pages=1):
    """Get all tracks from a list of playlist IDs"""
    
    all_tracks = [] 
    for id in playlist_ids:
        count = 0
        tracks = spotify.playlist_tracks(id, limit=limit_per_playlist)
        all_tracks.extend(tracks['items'])
        while tracks['next'] and count < pages:
            tracks = spotify.next(tracks)
            all_tracks.extend(tracks['items'])
            count += 1
    return all_trackshttp://localhost:8888/notebooks/Untitled.ipynb?kernel_name=python3#

In [24]:
ids = get_all_user_playlist_ids('spotify')
tracks = get_all_playlist_tracks(ids)
print(len(tracks))

2548


In [27]:
all_artists = []
for track in tracks:
    if 'track' in track and track['track'] != None:
        all_artists.extend(track['track']['artists'])
all_artist_ids = [artist['id'] for artist in all_artists]
all_artist_ids = pd.Series(all_artist_ids).drop_duplicates()
all_artist_ids = all_artist_ids.dropna()
artists = pd.Series(all_artist_ids, name='Artist_Id')
artists.to_csv('data/artist_ids_sample.csv', index=False)